Amalina Shabrina - M283B4KX0449

In [10]:
!pip install scikit-surprise # install the package

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 4.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357279 sha256=4e0cf80d28bbaea1e85ded9fa8f326906ada2e45c5a8e2ff78b80b444747ae9b
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [12]:
# Import library yang dibutuhkan
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from surprise import SVD, Dataset, Reader
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse
from datetime import datetime

# === Load Dataset ===
df_tourism_with_id = pd.read_csv('/content/drive/MyDrive/archive (1)/tourism_with_id.csv')  # Path ke tourism_with_id
df_tourism_rating = pd.read_csv('/content/drive/MyDrive/archive (1)/tourism_rating.csv')    # Path ke tourism_rating

# Gabungkan dataset untuk akses lebih mudah
df_merged = pd.merge(df_tourism_rating, df_tourism_with_id, on='Place_Id', how='inner')

# Tambahkan simulasi timestamp jika diperlukan
df_merged['Timestamp'] = pd.date_range(start='2023-01-01', periods=len(df_merged), freq='D')

In [14]:
# === Persiapan Model Machine Learning ===
# Preprocessing data untuk library Surprise
reader = Reader(rating_scale=(0, 5))  # Rating berkisar antara 0 hingga 5
data = Dataset.load_from_df(df_merged[['User_Id', 'Place_Id', 'Place_Ratings']], reader)

# Membagi dataset menjadi data latih dan uji
trainset, testset = train_test_split(data, test_size=0.2)

# Melatih model dengan SVD
model = SVD()
model.fit(trainset)

# Evaluasi model
predictions = model.test(testset)
rmse_value = rmse(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse_value}")

RMSE: 1.4274
Root Mean Squared Error (RMSE): 1.427390728892452


In [15]:
# === Fungsi Rekomendasi Pengguna Baru ===
def rekomendasi_pengguna_baru(kategori, lokasi=None, top_n=5):
    """
    Rekomendasi tempat wisata untuk pengguna baru berdasarkan kategori wisata dan lokasi.
    """
    # Filter berdasarkan kategori
    filtered_places = df_merged[df_merged['Category'].str.contains(kategori, case=False)]

    # Filter berdasarkan lokasi jika disediakan
    if lokasi:
        filtered_places = filtered_places[filtered_places['City'].str.contains(lokasi, case=False)]

    # Urutkan berdasarkan rating rata-rata tempat wisata
    rekomendasi = (
        filtered_places[['Place_Name', 'Category', 'City', 'Rating']]
        .drop_duplicates()
        .sort_values(by='Rating', ascending=False)
        .head(top_n)
    )
    return rekomendasi

In [16]:
# === Tambahkan Informasi Tren ===
# Hitung jumlah kunjungan untuk setiap tempat
trend_visits = df_merged.groupby('Place_Id').size().reset_index(name='Visit_Count')

# Hitung rata-rata rating terkini (6 bulan terakhir)
cutoff_date = datetime.now() - pd.Timedelta(days=180)
recent_ratings = df_merged[df_merged['Timestamp'] > cutoff_date]
trend_ratings = recent_ratings.groupby('Place_Id')['Place_Ratings'].mean().reset_index(name='Recent_Avg_Rating')

# Gabungkan informasi tren dengan dataset wisata
df_tourism_with_id = df_tourism_with_id.merge(trend_visits, on='Place_Id', how='left').merge(trend_ratings, on='Place_Id', how='left')
df_tourism_with_id['Visit_Count'] = df_tourism_with_id['Visit_Count'].fillna(0)
df_tourism_with_id['Recent_Avg_Rating'] = df_tourism_with_id['Recent_Avg_Rating'].fillna(0)



In [17]:
# === Fungsi Rekomendasi Pengguna Lama ===
def rekomendasi_pengguna_lama(user_id, top_n=5):
    """
    Memberikan rekomendasi untuk pengguna tertentu berdasarkan model SVD dan tren.
    """
    # Tempat yang sudah pernah dilihat oleh user
    seen_places = df_merged[df_merged['User_Id'] == user_id]['Place_Id'].unique()

    # Prediksi rating untuk tempat yang belum dilihat
    all_places = df_merged['Place_Id'].unique()
    rekomendasi = []
    for place_id in all_places:
        if place_id not in seen_places:
            pred = model.predict(user_id, place_id)
            rekomendasi.append((place_id, pred.est))

    # Buat DataFrame rekomendasi
    rekomendasi_df = pd.DataFrame(rekomendasi, columns=['Place_Id', 'Predicted_Rating'])

    # Gabungkan dengan data wisata dan informasi tren
    rekomendasi_df = rekomendasi_df.merge(df_tourism_with_id, on='Place_Id')

    # Tambahkan skor tren (kombinasi jumlah kunjungan dan rata-rata rating terkini)
    rekomendasi_df['Trend_Score'] = (
        rekomendasi_df['Visit_Count'] * 0.6 +  # Bobot jumlah kunjungan
        rekomendasi_df['Recent_Avg_Rating'] * 0.4  # Bobot rata-rata rating terkini
    )

    # Gabungkan skor prediksi dan tren
    rekomendasi_df['Final_Score'] = (
        rekomendasi_df['Predicted_Rating'] * 0.7 +  # Bobot prediksi rating
        rekomendasi_df['Trend_Score'] * 0.3  # Bobot tren
    )

    # Urutkan berdasarkan skor akhir dan pilih top_n
    rekomendasi_df = rekomendasi_df.sort_values(by='Final_Score', ascending=False).head(top_n)
    return rekomendasi_df[['Place_Id', 'Place_Name', 'Category', 'Predicted_Rating', 'Trend_Score', 'Final_Score']]


In [18]:
# === Contoh Penggunaan ===
# Rekomendasi untuk pengguna baru
kategori_wisata = "Budaya"
lokasi_wisata = "Yogyakarta"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                        Place_Name Category        City  Rating
582   Kauman Pakualaman Yogyakarta   Budaya  Yogyakarta     5.0
1323          Museum Ullen Sentalu   Budaya  Yogyakarta     4.7
446                Candi Borobudur   Budaya  Yogyakarta     4.7
314            Wisata Kraton Jogja   Budaya  Yogyakarta     4.7
163                Candi Prambanan   Budaya  Yogyakarta     4.7


In [20]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan lokasi
kategori_wisata = "Budaya"
lokasi_wisata = "Semarang"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                     Place_Name Category      City  Rating
94              Pura Giri Natha   Budaya  Semarang     4.7
141          Tugu Muda Semarang   Budaya  Semarang     4.7
527          Kota Lama Semarang   Budaya  Semarang     4.7
977  Kampoeng Djadhoel Semarang   Budaya  Semarang     4.7
739                 Lawang Sewu   Budaya  Semarang     4.6


In [21]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan tanpa lokasi
kategori_wisata = "Budaya"
lokasi_wisata = ""
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                          Place_Name Category        City  Rating
582     Kauman Pakualaman Yogyakarta   Budaya  Yogyakarta     5.0
1232                 Freedom Library   Budaya     Jakarta     5.0
324            Galeri Indonesia Kaya   Budaya     Jakarta     4.8
155   Museum Sasmita Loka Ahmad Yani   Budaya     Jakarta     4.7
314              Wisata Kraton Jogja   Budaya  Yogyakarta     4.7


In [22]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan tanpa lokasi
kategori_wisata = "Alam"
lokasi_wisata = ""
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                       Place_Name    Category      City  Rating
653  Kebun Tanaman Obat Sari Alam  Cagar Alam   Bandung     4.9
171                Gunung Lalakon  Cagar Alam   Bandung     4.8
732      Gua Maria Kerep Ambarawa  Cagar Alam  Semarang     4.8
520                 Curug Aseupan  Cagar Alam   Bandung     4.7
58                       Wot Batu  Cagar Alam   Bandung     4.7


In [23]:
# Rekomendasi untuk pengguna baru berdasarkan kategori dan lokasi
kategori_wisata = "Alam"
lokasi_wisata = "Yogyakarta"
print("Rekomendasi untuk pengguna baru:")
print(rekomendasi_pengguna_baru(kategori_wisata, lokasi=lokasi_wisata))

Rekomendasi untuk pengguna baru:
                                Place_Name    Category        City  Rating
725            Bukit Paralayang, Watugupit  Cagar Alam  Yogyakarta     4.7
337  Puncak Gunung Api Purba - Nglanggeran  Cagar Alam  Yogyakarta     4.7
11                      Taman Sungai Mudal  Cagar Alam  Yogyakarta     4.6
890                           Goa Jomblang  Cagar Alam  Yogyakarta     4.6
135              Seribu Batu Songgo Langit  Cagar Alam  Yogyakarta     4.6


In [19]:
# Rekomendasi untuk pengguna lama
user_id = 1
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=1:
     Place_Id                                 Place_Name       Category  \
213       177                        Pantai Parangtritis         Bahari   
12        437  Gereja Perawan Maria Tak Berdosa Surabaya  Tempat Ibadah   
113       298                             Gunung Lalakon     Cagar Alam   
241       201                             Pantai Kesirat         Bahari   
17        167              Geoforest Watu Payung Turunan     Cagar Alam   

     Predicted_Rating  Trend_Score  Final_Score  
213          3.516934    24.673684     9.863959  
12           3.398702    24.068571     9.599663  
113          3.052073    24.488889     9.483118  
241          2.964333    23.444444     9.108366  
17           3.454174    21.746667     8.941922  


In [24]:
# Rekomendasi untuk pengguna lama
user_id = 2
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=2:
     Place_Id                   Place_Name            Category  \
215       177          Pantai Parangtritis              Bahari   
115       298               Gunung Lalakon          Cagar Alam   
243       201               Pantai Kesirat              Bahari   
160       416             Keraton Surabaya              Budaya   
148       402  Food Junction Grand Pakuwon  Pusat Perbelanjaan   

     Predicted_Rating  Trend_Score  Final_Score  
215          3.133665    24.673684     9.595671  
115          2.622343    24.488889     9.182307  
243          2.836140    23.444444     9.018631  
160          3.755080    20.186667     8.684556  
148          3.063690    21.093333     8.472583  


In [25]:
# Rekomendasi untuk pengguna lama
user_id = 3
print(f"\nRekomendasi untuk pengguna lama dengan User_Id={user_id}:")
print(rekomendasi_pengguna_lama(user_id=user_id, top_n=5))


Rekomendasi untuk pengguna lama dengan User_Id=3:
     Place_Id                                 Place_Name       Category  \
213       177                        Pantai Parangtritis         Bahari   
40        437  Gereja Perawan Maria Tak Berdosa Surabaya  Tempat Ibadah   
241       201                             Pantai Kesirat         Bahari   
113       298                             Gunung Lalakon     Cagar Alam   
11        208                         Taman Sungai Mudal     Cagar Alam   

     Predicted_Rating  Trend_Score  Final_Score  
213          3.466897    24.673684     9.828933  
40           3.561488    24.068571     9.713613  
241          3.760389    23.444444     9.665606  
113          3.134660    24.488889     9.540929  
11           3.535161    22.896970     9.343703  


In [26]:
import pickle

# Menyimpan model SVD ke dalam file pickle
with open('svd_model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Model disimpan ke dalam file 'svd_model.pkl'")


Model disimpan ke dalam file 'svd_model.pkl'


In [27]:
# Memuat model SVD dari file pickle
with open('svd_model.pkl', 'rb') as f:
    loaded_model = pickle.load(f)

print("Model berhasil dimuat!")


Model berhasil dimuat!


In [28]:
from google.colab import files

# Menyimpan model ke dalam file pickle
with open('svd_model.pkl', 'wb') as f:
    pickle.dump(model, f)

# Mengunduh file pickle ke komputer lokal
files.download('svd_model.pkl')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>